In [1]:
#@title
!git clone https://github.com/facebookresearch/BenchMARL
#@title
%cd /content/BenchMARL
#@title
!pip install -U torch torchvision
!pip install -e .
#@title
!pip install dm-meltingpot
!apt-get update
!apt-get install -y x11-utils
!apt-get install -y xvfb
!apt-get install -y imagemagick
!pip install pyvirtualdisplay
!pip install id-marl-eval
!apt-get install python3-opengl
!pip install --upgrade chex
!pip install wandb
!pip install vmas[all]
!python -m pip install vmas
!python -c "import vmas; print(vmas.__version__)"
import pyvirtualdisplay
display = pyvirtualdisplay.Display(visible=False, size=(1400, 900))
display.start()

fatal: destination path 'BenchMARL' already exists and is not an empty directory.
/content/BenchMARL
Obtaining file:///content/BenchMARL
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for benchmarl (pyproject.toml) ... done
  Created wheel for benchmarl: filename=benchmarl-1.3.0-0.editable-py3-none-any.whl size=3799 sha256=90b7e6328f717916eb76632e0a2af3827ce7338baf31a345309bcc166a869fc5
  Stored in directory: /tmp/pip-ephem-wheel-cache-k1jokk0k/wheels/5b/86/4d/eff20c27275b75bdf6b20e4a81af849509b0068ff4c44be9df
Successfully built benchmarl
  Attempting uninstall: benchmarl
    Found existing installation: benchmarl 1.3.0
    Uninstalling benchmarl-1.3.0:
      Successfully uninstalled benchmarl-1.3.0
  Using cached chex-0.1.7-py3-none-any.whl.metadata (17 kB)
Using cached chex-0.1.7-py3-none-any.whl (89 kB)
 

In [2]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: 01019799 (01019799-marcingodniak). Use `wandb login --relogin` to force relogin


True

In [3]:
import os
from pathlib import Path
from typing import List

from benchmarl.eval_results import load_and_merge_json_dicts, Plotting
from matplotlib import pyplot as plt

/content/BenchMARL/benchmarl/environments/magent/common.py:45: DeprecationWarning: private variables, such as '_MAgentTask__get_env', will be normal attributes in 3.11
  def __get_env(self, config) -> EnvBase:
/usr/local/lib/python3.10/dist-packages/colorcet/__init__.py:82: MatplotlibDeprecationWarning: The register_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps.register(name)`` instead.
  register_cmap("cet_"+name, cmap=cm[name])


This function configures and returns the experiment setup by using the `ExperimentConfig` class from the `benchmarl` package. The configuration includes parameters like logging settings, iteration limits, and computational resources. We decided to override some of the default experiment parameters.


| **Argument**                        | **Type**           | **Description**                                                                                                                                                 |
|-------------------------------------|--------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `experiment_config.save_folder`     | `Path`             | Directory where the experiment data (such as logs, checkpoints) will be saved. By default, it is set to the current working directory (`Path.cwd()`).          |
| `experiment_config.loggers`         | `list of str`      | List of logging tools to be used during the experiment. In this case, the only logger specified is `"wandb"`, which stands for Weights & Biases.                |
| `experiment_config.max_n_iters`     | `int`              | Maximum number of iterations for the experiment. In this function, it is set to 200.                                                                          |
| `experiment_config.eval_frequency`  | `int`              | Defines the frequency (in terms of iterations) at which evaluation is done. Set to 5 iterations in this case.                                                  |
| `experiment_config.checkpoint_frequency` | `int`         | Defines how often (in terms of iterations) a checkpoint will be saved. Set to 2 iterations here.                                                              |
| `experiment_config.verbose`         | `bool`             | Whether to print detailed logs during the experiment. If `True`, it enables verbose output.                                                                   |
| `experiment_config.num_workers`     | `int`              | The number of workers (or threads) used for data loading and parallelization. Set to 4 in this case.                                                           |
| `experiment_config.device`          | `str`              | Specifies the device used for computation. Options are typically `"cuda"` (GPU) or `"cpu"`. Here, it is set to `"cuda"`, meaning the experiment will run on a GPU. |

In [4]:
def configure_experiment():
    """Configures and returns the experiment setup."""
    from benchmarl.experiment import ExperimentConfig
    from pathlib import Path

    # Create the experiment configuration
    experiment_config = ExperimentConfig.get_from_yaml()

    experiment_config.save_folder = Path.cwd()
    experiment_config.loggers = ["wandb"]
    experiment_config.max_n_iters = 200
    experiment_config.eval_frequency = 5
    experiment_config.checkpoint_frequency = 2
    experiment_config.verbose = False
    experiment_config.num_workers = 4
    experiment_config.device = "cuda"

    # Return the configured experiment
    return experiment_config

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Tasks Overview

This function configures and returns a list of tasks for the experiment setup.  Below are the brief descriptions of each task:

### Tasks:

1. **Simple Spread**  
   A cooperative task where agents must spread across landmarks while avoiding collisions.

2. **Simple Adversary**  
   A competitive task where agents try to reach a target landmark while avoiding an adversary.

3. **Navigation**  
   A light cooperative task where agents navigate to their goals while avoiding collisions.

4. **Simple World Comm**  
   A mixed task involving both cooperative and competitive elements, with agents avoiding adversaries and obstacles while interacting in a predator-prey setup.

Each task is configured using settings from the respective YAML containg default parameters specyfic to given task.


In [5]:
def configure_tasks():
    """Returns a list of configured tasks and prints their configurations."""
    from benchmarl.environments import VmasTask

    task_map = {
        'Simple Spread': VmasTask.SIMPLE_SPREAD,
        'Simple Adversary': VmasTask.SIMPLE_ADVERSARY,
        'Navigation': VmasTask.NAVIGATION,
        'Simple World Comm': VmasTask.SIMPLE_WORLD_COMM,
    }

    tasks = []
    for task_name, task_class in task_map.items():
        task = task_class.get_from_yaml()
        tasks.append((task_name, task))

    print("Tasks: ")
    for i, (task_name, task) in enumerate(tasks, start=1):
        print(f"Task {i}: {task_name}")
        print(f"Configuration: {task.config}")
        print("-" * 50)
    print("\n\n")

    return [task for _, task in tasks]

## Algorithms Overview

This function configures and returns a list of algorithms for the experiment setup. Below are brief descriptions of the two algorithms and the shared parameter values:

### Algorithms:

1. **ISAC (Independent Actor-Critic)**  
   A reinforcement learning algorithm where each agent has its own actor and critic. The critics operate independently, leading to higher policy entropy and more exploration, though coordination among agents may be more difficult.

2. **MASAC (Multi-Agent Soft Actor-Critic)**  
   A centralized version of the actor-critic algorithm, where a global critic models the interactions between all agents. MASAC tends to perform faster convergence and better coordination between agents compared to ISAC.

Both algorithms share the following configuration values:

### Shared Parameter Values:

| Parameter                          | Value                             | Description                                                                                   |
|------------------------------------|-----------------------------------|-----------------------------------------------------------------------------------------------|
| `share_param_critic`               | `True`                            | Whether the critic parameters are shared between agents.                                       |
| `num_qvalue_nets`                  | `2`                               | The number of Q-value networks used in the critic.                                             |
| `loss_function`                    | `'l2'`                            | The loss function used for training the Q-value networks (L2 loss in this case).               |
| `delay_qvalue`                     | `True`                            | Whether the Q-value update is delayed to avoid overfitting.                                    |
| `target_entropy`                   | `'auto'`                          | Target entropy used for adjusting exploration; `'auto'` allows automatic tuning.              |
| `discrete_target_entropy_weight`   | `0.2`                             | Weight applied to discrete target entropy in the reward function.                             |
| `alpha_init`                       | `1.0`                             | Initial value of the entropy regularization coefficient (`alpha`).                             |
| `min_alpha`                        | `None`                            | Minimum value of `alpha` to avoid too much regularization.                                     |
| `max_alpha`                        | `None`                            | Maximum value of `alpha` to avoid too little regularization.                                  |
| `fixed_alpha`                      | `False`                           | Whether to fix the value of `alpha` during training.                                           |
| `scale_mapping`                    | `'biased_softplus_1.0'`           | The function used to map actions to the desired scale during training (biased softplus mapping).|
| `use_tanh_normal`                  | `True`                            | Whether to use Tanh-normal distribution for sampling actions.                                  |

Both ISAC and MASAC are configured using settings from their respective YAML files and can be further customized for specific use cases.


In [6]:
def configure_algorithms():
    """Returns a list of configured algorithm configurations and prints their names and configurations."""
    from benchmarl.algorithms import IsacConfig, MasacConfig

    # Define the algorithms and their names
    algorithms = [
        ('ISAC', IsacConfig.get_from_yaml()),
        ('MASAC', MasacConfig.get_from_yaml()),
    ]

    # Print the name and configuration of each algorithm
    print("Algorithms: ")
    for i, (algo_name, algo_config) in enumerate(algorithms, start=1):
        print(f"Algorithm {i}: {algo_name}")
        print(f"Configuration: {algo_config}")  # Access configuration using .config
        print("-" * 50)
    print("\n\n")

    return [algo_config for _, algo_config in algorithms]

### Model and Critic Configuration

In this setup, both the **actor** and the **critic** will use a **Multi-Layer Perceptron (MLP)** architecture. An MLP is a type of feedforward neural network that is commonly used for approximating functions in reinforcement learning tasks.

- **Actor Model**: The actor uses the MLP to decide on actions based on the current state of the environment.
- **Critic Model**: The critic also uses the MLP to estimate the value of the actions taken by the actor, providing feedback for improvement.

| **Parameter**          | **Description**                                                                                                                                                           |
|------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `num_cells`            | A list specifying the number of neurons in each layer of the MLP. In this case, the MLP consists of two layers, each with 256 neurons.                                     |
| `layer_class`          | Defines the type of layer used in the MLP. Here, it is a `Linear` layer, which is a fully connected layer.                                                               |
| `activation_class`     | Specifies the activation function used between layers. In this case, `Tanh` is applied, which is a common activation function for neural networks.                        |

In [7]:
def configure_models():
    """Returns the model and critic model configurations."""
    from benchmarl.models.mlp import MlpConfig

    model_config = MlpConfig.get_from_yaml()         # Actor model configuration
    critic_model_config = MlpConfig.get_from_yaml()  # Critic model configuration

    print("Model Configuration:")
    print(model_config)
    print("Critic Model Configuration:")
    print(critic_model_config)

    return model_config, critic_model_config


In [8]:
def run_benchmark() -> List[str]:
    """Runs the benchmark and returns a list of experiment JSON file paths."""
    from benchmarl.benchmark import Benchmark

    # Configure experiment, tasks, algorithms, and models
    experiment_config = configure_experiment()
    tasks = configure_tasks()
    algorithm_configs = configure_algorithms()
    model_config, critic_model_config = configure_models()

    # Initialize benchmark
    benchmark = Benchmark(
        algorithm_configs=algorithm_configs,
        tasks=tasks,
        seeds={0},
        experiment_config=experiment_config,
        model_config=model_config,
        critic_model_config=critic_model_config,
    )

    # Run experiments
    experiments = benchmark.get_experiments()
    experiments_json_files = []
    for experiment in experiments:
        exp_json_file = str(
            Path(experiment.folder_name) / Path(experiment.name + ".json")
        )
        experiments_json_files.append(exp_json_file)
        experiment.run()
    return experiments_json_files

In [9]:
def process_results(experiments_json_files: List[str]):
    """Processes experiment results and generates comparison matrices."""
    # Load and process experiment outputs
    raw_dict = load_and_merge_json_dicts(experiments_json_files)
    processed_data = Plotting.process_data(raw_dict)
    (
        environment_comparison_matrix,
        sample_efficiency_matrix,
    ) = Plotting.create_matrices(processed_data, env_name="vmas")
    return processed_data, environment_comparison_matrix, sample_efficiency_matrix

In [10]:
def generate_plots(processed_data, environment_comparison_matrix, sample_efficiency_matrix):
    """Generates and saves all required plots."""
    # Performance profile plot
    Plotting.performance_profile_figure(
        environment_comparison_matrix=environment_comparison_matrix
    )
    plt.savefig("performance.png")

    # Aggregate scores plot
    Plotting.aggregate_scores(
        environment_comparison_matrix=environment_comparison_matrix
    )
    plt.savefig("aggregate_scores.png")

    # Sample efficiency curves
    Plotting.environemnt_sample_efficiency_curves(
        sample_effeciency_matrix=sample_efficiency_matrix
    )
    plt.savefig("sample_efficiency.png")

    # Task-specific sample efficiency curves for Simple Spread and Simple Adversary
    Plotting.task_sample_efficiency_curves(
        processed_data=processed_data, env="vmas", task="simple_spread"
    )
    plt.savefig("task_sample_efficiency_spread.png")

    Plotting.task_sample_efficiency_curves(
        processed_data=processed_data, env="vmas", task="simple_adversary"
    )
    plt.savefig("task_sample_efficiency_adversary.png")

    # Task-specific sample efficiency curves for Navigation and Simple World Comm
    Plotting.task_sample_efficiency_curves(
        processed_data=processed_data, env="vmas", task="navigation"
    )
    plt.savefig("task_sample_efficiency_navigation.png")

    Plotting.task_sample_efficiency_curves(
        processed_data=processed_data, env="vmas", task="simple_world_comm"
    )
    plt.savefig("task_sample_efficiency_world_comm.png")

    # Display all plots
    plt.show()


In [ ]:
# Run benchmark and get JSON files
experiments_json_files = run_benchmark()

# Process results
processed_data, environment_comparison_matrix, sample_efficiency_matrix = process_results(experiments_json_files)

# Generate plots
generate_plots(
    processed_data, environment_comparison_matrix, sample_efficiency_matrix
)


Tasks: 
Task 1: Simple Spread
Configuration: {'max_steps': 100, 'obs_agents': True, 'n_agents': 3}
--------------------------------------------------
Task 2: Simple Adversary
Configuration: {'max_steps': 100, 'n_agents': 3, 'n_adversaries': 1}
--------------------------------------------------
Task 3: Navigation
Configuration: {'max_steps': 100, 'n_agents': 3, 'collisions': True, 'agents_with_same_goal': 1, 'observe_all_goals': False, 'shared_rew': False, 'split_goals': False, 'lidar_range': 0.35, 'agent_radius': 0.1}
--------------------------------------------------
Task 4: Simple World Comm
Configuration: {'max_steps': 100, 'num_good_agents': 2, 'num_adversaries': 4, 'num_landmarks': 1, 'num_food': 2, 'num_forests': 2}
--------------------------------------------------



Algorithms: 
Algorithm 1: ISAC
Configuration: IsacConfig(share_param_critic=True, num_qvalue_nets=2, loss_function='l2', delay_qvalue=True, target_entropy='auto', discrete_target_entropy_weight=0.2, alpha_init=1.0,

/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_init.py:202: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `include` and `exclude`. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  settings = self._wl.settings.copy()


  0%|          | 0/10 [00:00<?, ?it/s]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/fx/painting.py:7: DeprecationWarning: Please import `sobel` from the `scipy.ndimage` namespace; the `scipy.ndimage.filters` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.filters import sobel

mean return = -524.8851928710938: 100%|██████████| 10/10 [02:53<00:00, 17.14s/it]

collection/agents/reward/episode_reward_max,▇▄▂▂▁█▂▄▅▅
collection/agents/reward/episode_reward_mean,▃▄▃▃▄▁▆▄▅█
collection/agents/reward/episode_reward_min,▇▆▁▄▁▆▅█▂▇
collection/reward/episode_reward_max,▇▄▂▂▁█▂▄▅▅
collection/reward/episode_reward_mean,▃▄▃▃▄▁▆▄▅█
collection/reward/episode_reward_min,▇▆▁▄▁▆▅█▂▇
collection/reward/reward_max,▅▇▃▂▁▇█▆▅▄
collection/reward/reward_mean,▃▄▃▃▄▁▆▄▅█
collection/reward/reward_min,▆▆▂▁▂▅██▁▄
counters/current_frames,▁▁▁▁▁▁▁▁▁▁
counters/iter,▁▂▃▃▄▅▆▆▇█


  lambda data: self._console_raw_callback("stderr", data),

mean return = -524.8851928710938: 100%|██████████| 10/10 [02:56<00:00, 17.62s/it]
  settings = self._wl.settings.copy()



...

  lambda data: self._console_raw_callback("stderr", data),

mean return = -41.256187438964844: 100%|██████████| 10/10 [05:04<00:00, 30.47s/it]
  settings = self._wl.settings.copy()



mean return = -0.23322419822216034: 100%|██████████| 10/10 [03:02<00:00, 18.01s/it]

collection/agents/info/agent_collisions,█▆▆▇▆▆▇▄▁▆
collection/agents/info/final_rew,▁▁▁▁▁▁▁▁▁▁
collection/agents/info/pos_rew,▁▆▆▅▄▄▇██▆
collection/agents/reward/episode_reward_max,▁▃▄█▁▂▄▂▅▄
collection/agents/reward/episode_reward_mean,█▇▆▇▆▆█▅▁▇
collection/agents/reward/episode_reward_min,█▄▇▅▆▅▇▂▁▅
collection/reward/episode_reward_max,▁▃▄█▁▂▄▂▅▄
collection/reward/episode_reward_mean,█▇▆▇▆▆█▅▁▇
collection/reward/episode_reward_min,█▄▇▅▆▅▇▂▁▅
collection/reward/reward_max,▇▆▄█▄▄▄▆▁▄
collection/reward/reward_mean,█▇▆▇▆▆█▅▁▇


  lambda data: self._console_raw_callback("stderr", data),

mean return = -0.23322419822216034: 100%|██████████| 10/10 [03:04<00:00, 18.49s/it]
  settings = self._wl.settings.copy()



mean return = 10.188268661499023:  10%|█         | 1/10 [01:01<09:12, 61.41s/it]

## Report

### Analysis reproduction

The analysys was computed using Google Colab environment within the notebook. The wandb account is required for logging purposes.

### Pre-result discussion

The original conception has leaned towards the tsask from a MeltingPot environment. However, during testing process, the CNN networks representing Actors and Critics necessary to process visual environment provided by a task has proven to be too costly in computation power to acquire substantive results. Therfore, the decision has been made to pivot towards [VMAS](https://github.com/proroklab/VectorizedMultiAgentSimulator) environment with more simple tasks that support model architecture on level of MLP. Tasks and upcoming performance of the algorithms has been reported below.

### Task: Simple adversary (Adversary)
#### Description
1 adversary (red), N good agents (green), N landmarks (usually N=2). All agents observe position of landmarks and other agents. One landmark is the ‘target landmark’ (colored green). Good agents rewarded based on how close one of them is to the target landmark, but negatively rewarded if the adversary is close to target landmark. Adversary is rewarded based on how close it is to the target, but it doesn’t know which landmark is the target landmark. So good agents have to learn to ‘split up’ and cover all landmarks to deceive the adversary.

#### Parametrization
The task has been parametrized using hydra as:

```yaml
max_steps: 100
n_agents: 3
n_adversaries: 1
```

#### ISAC vs MASAC
##### Training
![](https://github.com/RichardStaszkiewicz/WUT-USD-MASCvsISAC/blob/main/graphics/adversary-train1.png?raw=1)
![](https://github.com/RichardStaszkiewicz/WUT-USD-MASCvsISAC/blob/main/graphics/adversary-train2.png?raw=1)

There are two key aspects we should pay attention during training: (1) Coordination, (2) Exploration. Our expectations evolve around ISAC with independent critics having more troubles to learn to coordinate agents efforts, but discovering more varied policies. On the other hand the MASAC with critic covering global interactions should have less trouble with coordination while performing faster convergence.

Based on a logs above we can observe that the *train/agent/entropy* does support the claim of ISAC's higher policy entropy, however the convergence speed as shown on agent and adversary loss Q-values do not make it look like a MSAC is converging significantly faster, quite the opposit - MSAC show tendences to initially plateu on a worse values then ISAC and only in remaining time slowly converge to around the end outperform individual critics. This might be a property of a costly general critic training, that however if trained beter generalises the environment then an agent-specified ones. What should be pointed out is that although the Q-value does not converge, the entropy influence parameter alpha does indeed in case of MSAC fall faster along with the agent entropy.

##### Evaluation
![](https://github.com/RichardStaszkiewicz/WUT-USD-MASCvsISAC/blob/main/graphics/aversary-eval.png?raw=1)

During evaluation one can observe the ISAC algorithm performing with higher variety in minimal and maximal reward of used policies due to the higher entropy of individual learning with low degree of effort coordination.

### Task: Simple Spread (Cooperation)
#### Description
N agents, N landmarks. Agents are rewarded based on how far any agent is from each landmark. Agents are penalized if they collide with other agents. So, agents have to learn to cover all the landmarks while avoiding collisions.

#### Parametrization
The task has been parametrized using hydra as:
```yaml
max_steps: 100
n_agents: 3
obs_agents: True
```
The agent observability has been added to make learning the patterns easier using the other agents positions in a frame of reference.

#### ISAC vs MASAC
##### Training
![](https://github.com/RichardStaszkiewicz/WUT-USD-MASCvsISAC/blob/main/graphics/spread-train.png?raw=1)

In this situation, the expected consequence of a global vs independend critic struggle is collision learning: the MASAC is set up to jointly learn collision avoidance while ISAC will have to independently learn paths of the others. The normalized loss gradients of both critic and actor in MASAC show signinficantly higher values, presumably due to high awarness and therefore incorporation of the collision penalty. One can also observe the expected Q-value (critic) is more reliable within centralized critic (MASAC), probably because of the collision loss effective modeling. Entropy supports the claim with ISAC behaving with higher variance without a centralized policy verification.

##### Evaluation
![](https://github.com/RichardStaszkiewicz/WUT-USD-MASCvsISAC/blob/main/graphics/spread-eval.png?raw=1)

As shown during evaluation, the MASAC has proven to take sizable lead in learning the cooperative task. After the 250 steps, it has reliably been delivering better mean results then ISAC, have also mostly been outperforming the other algorithm within minimum reward (except some hicup on step 400 evaluation - probably a collision penalty). Interesting part that could be observed is relative stability of the maximal rewards earned by the algorithms, as they do not incorporate mostly the very swingy collision penalties: worth pointing out is a relative competitivness of both algorithms on steps between 150 and 300. This moment in time could probably be the one, where global MASAC critic hasn't learned to avoid collisions reliably yet while ISAC independent critics were able to optimize the attraction fields coverage navigation.

### Task: Navigation (very light coop)

#### Description
Randomly spawned agents need to navigate to their goal. Collisions can be turned on and agents can use LIDARs to avoid running into each other. Rewards were turned to be shared in form of L2 distance. Apart from position, velocity, and lidar readings, each agent can be set up to observe just the relative distance to its goal, or its relative distance to all goals (in this case the task needs heterogeneous behavior to be solved).

#### Parametrization
The task has been parametrized using hydra as:
```yaml
max_steps: 100
n_agents: 3
collisions: True
agents_with_same_goal: 1
split_goals: False
observe_all_goals: False
shared_rew: False
lidar_range: 0.35
agent_radius: 0.1
```
The task is cooperative in terms of collisions, as agents are trying to avoid those. However as the rewards are not shared and the goals are not split, it is a very light interaction as for a multi-agent environment.

#### ISAC vs MASAC
##### Training
![](https://github.com/RichardStaszkiewicz/WUT-USD-MASCvsISAC/blob/main/graphics/navigation-train.png?raw=1)

Due to incorporation of collision penalties in global MASAC critic, the agents of this algorithm show slower convergence and high entropy. What is interesting, the high entropy was expected to be shown during ISAC analysis, but apparently the dirastic changes to a learning central critic has forced individual actors to adapt on the fly. In stage between 100-150 steps the interesting behaviour is exhibited by an entropy where agents entropy has polarized from around 1 to -1 as their policies during training emerged. ISAC was the first algorithm to start optimizing around entropy, while MASAC was still presumably trying to model the collision influence. Overall, the training concluded with MASAC having marginally smaller losses on its actors.


##### Evaluation
![](https://github.com/RichardStaszkiewicz/WUT-USD-MASCvsISAC/blob/main/graphics/navigation-eval.png?raw=1)

The evaluation has clearly identified the objectives different algorithms pursuit. By the episode length mean, we can learn it was in fact ISAC that learned to actually FINISH the task it was given - transport all the agents to their attraction fields, while MASAC was never able to compleate the task and was cut off on the episode 100. However, all the other statistics show clear supriority of the MASAC outcomes, even if it was not able to reach objective, the ISAC means were imposing too much losses. By the step 100, the MSAC confidently outperformed ISAC in reward collection in all fields: min, mean and max. Thus it could have been learned that paramisation of the environment had clear restrictions on collisions, to which central critic was able to adapt much better then the individuals.

### Task: Simple World Comm (Mixed)

#### Description
Predator-prey environment. Good agents (green) are faster and want to avoid being hit by adversaries (red). Adversaries are slower and want to hit good agents. Obstacles (large black circles) block the way. Moreover, (1) there is food (small blue balls) that the good agents are rewarded for being near, (2) we now have ‘forests’ that hide agents inside from being seen from outside; (3) there is a ‘leader adversary” that can see the agents at all times, and can communicate with the other adversaries to help coordinate the chase.

#### Parametrization
The task has been parametrized using hydra as:
```yaml
max_steps: 100
num_good_agents: 2
num_adversaries: 4
num_landmarks: 1
num_food: 2
num_forests: 2
```

#### ISAC vs MASAC
##### Training
![](https://github.com/RichardStaszkiewicz/WUT-USD-MASCvsISAC/blob/main/graphics/world-train1.png?raw=1)
![](https://github.com/RichardStaszkiewicz/WUT-USD-MASCvsISAC/blob/main/graphics/world-train2.png?raw=1)
![](https://github.com/RichardStaszkiewicz/WUT-USD-MASCvsISAC/blob/main/graphics/world-train3.png?raw=1)
As shown on above panels, both algorithms did have same learning rates mechanisms as gradients and alpha seems to remain same. [TODO]()
##### Evaluation
![](https://github.com/RichardStaszkiewicz/WUT-USD-MASCvsISAC/blob/main/graphics/world-eval.png?raw=1)
![](https://github.com/RichardStaszkiewicz/WUT-USD-MASCvsISAC/blob/main/graphics/world-eval2.png?raw=1)

### Discussion

The whole research was mostly boiled down to a **centralized (MASAC) vs. independedt (ISAC) critic**. The centralized version allows the critic to better model the environment's dynamics and the interaction between agents and adversaries, however is more costly in computation.



the final comparison table